In [2]:
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Sample data (replace with your actual data)
pop = pd.read_csv('data/songs_liwc_25_03_13.csv')

In [3]:
# processing: currently leaving punctuation alone
# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess

pop['lyrics'] = pop['lyrics'].astype(str).apply(remove_stopwords)
pop['lyrics-tokenized'] = pop['lyrics'].apply(simple_preprocess)


In [4]:
import torch

if torch.backends.mps.is_available():
    print("✅ MPS (Apple GPU) is available!")
else:
    print("❌ MPS not available. Running on CPU.")

✅ MPS (Apple GPU) is available!


In [ ]:
# emotion detection with distelroberta
# https://huggingface.co/j-hartmann/emotion-english-distilroberta-base
from transformers import pipeline
emotion_model = pipeline("text-classification", 
                         model="j-hartmann/emotion-english-distilroberta-base",
                         truncation = True)

pop['emotion'] = pop['lyrics-tokenized'].apply(lambda text: emotion_model(text)[0]['label'])

In [24]:
# add NRC
#https://www.geeksforgeeks.org/emotion-classification-using-nrc-lexicon-in-python/
from nrclex import NRCLex

#google ai popup:
def analyze_emotion(text):
    if isinstance(text, str):
        emotion_lexicon = NRCLex(text)
        return emotion_lexicon.raw_emotion_scores
    else:
        return {} 

#pop = pop.join(pop['emotion_scores'].apply(pd.Series))
pop['emotion_scores'] = pop['lyrics'].apply(analyze_emotion)

In [27]:
pop = pop.join(pop['emotion_scores'].apply(pd.Series))

In [28]:
pop.columns

Index(['Unnamed: 0', 'index', 'artist', 'album', 'song', 'lyrics', 'Segment',
       'WC', 'Analytic', 'Clout',
       ...
       'trust', 'anticipation', 'fear', 'surprise', 'positive', 'negative',
       'joy', 'sadness', 'anger', 'disgust'],
      dtype='object', length=137)

In [29]:
# work on PCA -- have distilbert, NRC + LIWC

pop.to_csv('pop_expanded_25_04_01.csv', index=False)

In [ ]:
# now LIWC, NRC, distilbert all together : time for PCA